# **TABELA FILIAL**

## <u>Realizando as transformações necessárias e criando nova tabela para ser consumida</u>

- Criaremos a coluna `Ordem` para ordernar as filiais uma vez que a PK da tabela staging é do tipo texto. 
- Além disso, preencheremos as informações nulas com informação da JC Brothers

In [3]:
CREATE OR ALTER PROCEDURE SP_CARREGA_FILIAL
	@CODFILIAL NCHAR(2),
	@EMPRESA NVARCHAR(150),
	@FILIAL NVARCHAR(25)
AS
BEGIN    
----Faz a transformação das informações
	DECLARE @ORDEM SMALLINT = CONVERT(SMALLINT, @CODFILIAL)
	DECLARE @DTUPDATE DATETIME = DATEADD(hour, -3, GETUTCDATE())

	SET @EMPRESA  = CASE WHEN @EMPRESA = 'NULL' THEN 'JC BROTHERS' ELSE @EMPRESA END
	SET @FILIAL  =  CASE WHEN @FILIAL = 'NULL' THEN  'JC BROTHERS' ELSE @FILIAL END
	
----faz o merge das informações
	IF EXISTS (SELECT 1 FROM FILIAL WHERE [CodFilial] = @CODFILIAL)
		BEGIN
		UPDATE FILIAL SET EMPRESA = @EMPRESA, FILIAL = @FILIAL, DTUPDATE = @DTUPDATE WHERE [CodFilial] = @CODFILIAL
	END
	ELSE
	BEGIN
		INSERT INTO FILIAL ([CodFilial], Empresa, Filial, Ordem, DtUpdate)
		VALUES (@CODFILIAL, @EMPRESA, @FILIAL, @ORDEM, @DTUPDATE)
	END
END

Commands completed successfully.

Total execution time: 00:00:00.039

In [1]:
SELECT * FROM FILIAL

(14 rows affected)

Total execution time: 00:00:00.073

CodFilial,Empresa,Filial,Ordem,DtUpdate
1,JC BROTHERS,LOJA JABAQUARA,1,2024-03-02 07:36:25.857
10,FFJ CORP,FFJ CORP,10,2024-03-02 07:36:17.933
11,JC BROTHERS,DISTRIBUICAO ES,11,2024-03-02 07:36:37.647
12,JC BROTHERS,LOJA JUNDIAI,12,2024-03-02 07:35:58.450
13,JC BROTHERS,LOJA TRIMAIS,13,2024-03-02 07:35:46.670
2,JC BROTHERS,DISTRIBUICAO SP,2,2024-03-02 07:36:02.280
3,JC BROTHERS,ESTOQUE CORPORATIVO,3,2024-03-02 07:36:29.717
4,JC BROTHERS,ESTOQUE SHOWROOM,4,2024-03-02 07:36:21.997
5,JFF,JFF CORP,5,2024-03-02 07:36:06.107
6,BROKER CORP,BROKER CORP,6,2024-03-02 07:36:10.170


In [7]:
CREATE OR ALTER PROCEDURE SP_RETORNA_FILIAL
    @CODFILIAL NCHAR(2)
AS
BEGIN    

    IF EXISTS (SELECT 1 FROM FILIAL WHERE CODFILIAL = @CODFILIAL)
	SELECT FORMAT(DTUPDATE, 'yyyy-MM-dd HH:mm:ss') DTUPDATE FROM FILIAL WHERE CODFILIAL = @CODFILIAL
END

Commands completed successfully.

Total execution time: 00:00:00.028

In [1]:
BEGIN
    EXEC SP_RETORNA_FILIAL '13'
END

(1 row affected)

Total execution time: 00:00:00.203

DTUPDATE
2024-03-04 19:05:27
